# Preprocesamiento de audios
## Adrián Arnaiz Rodríguez
### Eliminamos el silencio inicial y final de los audios para que no den medidas erróneas en el script prosody.py de disvoice. Explicación:

*En el caso de las medidas de prosodia se pueden tener algunos incovenientes si los silencios iniciales son muy largos puesto que tendríamos resultados erróneos en las caracteristicas relacionadas con la duracion promedio de silencios, la variabilidad de la duracion de los silencios y otras medidas que se calculan sobre las pausas.*

*Para los demás scripts (phonation.py y articulation.py) se utilizarán los audios originales. En el caso de phonation y articulation no es necesario eliminar estos silencios. Las caracteristicas de fonación se calculan solo sobre segmentos voiced, los cuales se segmentan automaticamente usando un script de praat (linea 138 de phonation.py, metodo paat_vuv de praat_functions.py). Igualmente para las de articulación, puesto que solo estan basadas en segmentos de transiciones de voiced a unvoiced, o vicerversa (segmentos onset u offset, lineas 175 y 176 de articulation.py).*

### 1. Importamos pydub para eliminar silencios

In [1]:
from pydub import AudioSegment

C:\Users\usuario\Anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


### 2. Creamos una función para que detecte los segmentos silenciosos al principio del audio
 1. Le pasamos un audio de tipo pydub.AudioSegment
 2. Establecemos unos dB límite para considerar qué es silencio
 3. Establecemos el tamaño del segmento (def=10ms)

In [2]:
def detect_leading_silence(sound, silence_threshold=-65.0, chunk_size=10):
    '''
    sound es del tipo pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

### 3. Usamos la funcióin anterior para eliminar silencio inicial y final y crear nuevo fichero de audio
Este fichero de audio tendrá el mismo nombre que el anterior pero acabado en \_S: audio.wav -> audio_S.wav

In [3]:
from os import listdir, scandir
def eliminar_silencios(ruta):
    '''
    Eliminamos los silencios inicial y final de los audios contenidos en una ruta concreta
    '''
    for audio in listdir(ruta):
        sound = AudioSegment.from_file(ruta+'/'+audio, format="wav")

        start_trim = detect_leading_silence(sound)
        end_trim = detect_leading_silence(sound.reverse())

        duration = len(sound)    
        trimmed_sound = sound[start_trim:duration-end_trim]
        trimmed_sound.export(audio[:-4]+'_S.wav', format="wav")


#### Ejecutamos un ejemplo

In [4]:
eliminar_silencios('PC-GITA/read-text/pd')

In [5]:
listdir()

['.ipynb_checkpoints',
 'AVPEPUDEA0001_readtext_S.wav',
 'AVPEPUDEA0002_readtext_S.wav',
 'AVPEPUDEA0003_readtext_S.wav',
 'AVPEPUDEA0005_readtext_S.wav',
 'AVPEPUDEA0006_readtext_S.wav',
 'AVPEPUDEA0007_readtext_S.wav',
 'AVPEPUDEA0008_readtext_S.wav',
 'AVPEPUDEA0009_readtext_S.wav',
 'AVPEPUDEA0010_readtext_S.wav',
 'AVPEPUDEA0011_readtext_S.wav',
 'AVPEPUDEA0013_readtext_S.wav',
 'AVPEPUDEA0014_readtext_S.wav',
 'AVPEPUDEA0015_readtext_S.wav',
 'AVPEPUDEA0016_readtext_S.wav',
 'AVPEPUDEA0017_readtext_S.wav',
 'AVPEPUDEA0020_readtext_S.wav',
 'AVPEPUDEA0021_readtext_S.wav',
 'AVPEPUDEA0022_readtext_S.wav',
 'AVPEPUDEA0023_readtext_S.wav',
 'AVPEPUDEA0024_readtext_S.wav',
 'AVPEPUDEA0025_readtext_S.wav',
 'AVPEPUDEA0026_readtext_S.wav',
 'AVPEPUDEA0027_readtext_S.wav',
 'AVPEPUDEA0029_readtext_S.wav',
 'AVPEPUDEA0030_readtext_S.wav',
 'AVPEPUDEA0031_readtext_S.wav',
 'AVPEPUDEA0032_readtext_S.wav',
 'AVPEPUDEA0034_readtext_S.wav',
 'AVPEPUDEA0035_readtext_S.wav',
 'AVPEPUDEA0037_read

In [6]:
!del *_S.wav #Borramos los audios del ejemplo